In [1]:
# https://keras.io/applications/#inceptionv3
import numpy as np

import keras

from keras.models import load_model
from keras_applications import inception_v3
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D

decode_predictions = inception_v3.decode_predictions
preprocess_input = inception_v3.preprocess_input

Using TensorFlow backend.


In [4]:
base_model = keras.applications.inception_v3.InceptionV3(include_top=True, 
                                                    weights='imagenet',
                                                    input_shape=(299, 299, 3))

In [6]:
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 149, 149, 32) 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, 149, 149, 32) 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [5]:
base_model.save('base.h5')

## Load Inception V3 without weight

In [28]:
random_model = keras.applications.inception_v3.InceptionV3(include_top=True, 
                                                          weights=None,
                                                          input_shape=(299,299,3))

In [22]:
# run a prediction on the model withot training
img_path = './train/shark/sharks2.png'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = random_model.predict(x)
print('Predicted:', decode_predictions(preds))

('Predicted:', [[(u'n03179701', u'desk', 0.0042196936), (u'n04254680', u'soccer_ball', 0.0034656876), (u'n02869837', u'bonnet', 0.0034430677), (u'n02127052', u'lynx', 0.0030195264), (u'n01498041', u'stingray', 0.0026553636)]])


In [27]:
# a fake example
x_train = np.random.normal(loc=127, scale=127, size=(50, 229,229,3))
y_train = np.array([0,1]*25)
x_train = preprocess_input(x_train)

print x_train.shape
print y_train.shape

(50, 229, 229, 3)
(50,)


### Pick-up training using new pictures

In [29]:
random_model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['acc'])

In [32]:
# load image and create a label
img_path = './train/shark/sharks2.png'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

label = np.array([3])
print label

[3]


In [33]:
print x.shape, label.shape

(1, 299, 299, 3) (1,)


In [34]:
# train
random_model.fit(x, label, epochs=3, batch_size=1, verbose=2)

Epoch 1/3
 - 21s - loss: 7.1971 - acc: 0.0000e+00
Epoch 2/3
 - 1s - loss: 7.0153 - acc: 0.0000e+00
Epoch 3/3
 - 1s - loss: 6.8182 - acc: 0.0000e+00


In [35]:
# predict the same image use the updated random model
preds = random_model.predict(x)
print('Predicted:', decode_predictions(preds))

('Predicted:', [[(u'n01692333', u'Gila_monster', 0.004310805), (u'n02490219', u'marmoset', 0.0042495313), (u'n02107683', u'Bernese_mountain_dog', 0.004238896), (u'n03841143', u'odometer', 0.0037095207), (u'n02841315', u'binoculars', 0.003545996)]])


In [44]:
# didn't work out well, how about on a pre-trained model, but I have to know how it get the compiled

# this is a model with weight
pretrained_model = keras.applications.inception_v3.InceptionV3(include_top=True, 
                                                          weights='imagenet',
                                                          input_shape=(299,299,3))
pretrained_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.0001),
              metrics=['acc'])

In [47]:
# load a new shark picture and predict on it

img_path = './train/shark.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = pretrained_model.predict(x)
print('Predicted:', decode_predictions(preds))

('Predicted:', [[(u'n01491361', u'tiger_shark', 0.8138281), (u'n01494475', u'hammerhead', 0.113477096), (u'n01484850', u'great_white_shark', 0.03277753), (u'n04552348', u'warplane', 0.0018323737), (u'n04008634', u'projectile', 0.0013928588)]])


In [66]:
# now, load this picture again, make up a label, and train the pre-trained model
img_path = './train/shark.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

label = np.zeros((1,1000))
label[0][3] = 1.0
preds.shape

(1, 1000)

In [67]:
pretrained_model.fit(x, label, epochs=3, batch_size=1, verbose=2)

Epoch 1/3
 - 25s - loss: 6.9234 - acc: 0.0000e+00
Epoch 2/3
 - 1s - loss: 6.4545 - acc: 1.0000
Epoch 3/3
 - 1s - loss: 5.9737 - acc: 1.0000


In [69]:
# run the prediction on the same picture again
# hopefully it won't mess-up the model weights
img_path = './train/cat/cat.1000.jpg'
img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = pretrained_model.predict(x)
print('Predicted:', decode_predictions(preds))

('Predicted:', [[(u'n03250847', u'drumstick', 0.7862794), (u'n03249569', u'drum', 0.108679965), (u'n04286575', u'spotlight', 0.058457434), (u'n03887697', u'paper_towel', 0.024892315), (u'n03633091', u'ladle', 0.004455708)]])
